In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sqlite3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os

cnx = sqlite3.connect('spotify.sqlite')
#cnx.text_factory = bytes
cnx.text_factory = lambda b: b.decode(errors = 'ignore')

In [3]:
df_tracks = pd.read_sql_query("SELECT id, name, preview_url, popularity, track_number, explicit FROM tracks", cnx)
df_tracks = df_tracks[df_tracks.preview_url !='']
df_albums = pd.read_sql_query("SELECT id, name FROM albums", cnx)
df_artists = pd.read_sql_query("SELECT id, name FROM artists", cnx)
df_albums_tracks = pd.read_sql_query("SELECT track_id, album_id FROM r_albums_tracks", cnx)
df_track_artist = pd.read_sql_query("SELECT track_id, artist_id FROM r_track_artist", cnx)

df_artist_genre = pd.read_sql_query("SELECT * FROM r_artist_genre", cnx)
df_audio_features = pd.read_sql_query("SELECT * FROM audio_features", cnx)

In [4]:
df_tracks = df_tracks.rename(columns={'name': 'track', 'id': 'track_id'})
df_albums = df_albums.rename(columns={'name': 'album', 'id': 'album_id'})
df_artists = df_artists.rename(columns={'name': 'artist', 'id': 'artist_id'})

df_albums_tracks = df_albums_tracks.rename(columns={'album_id': 'album_id', 'track_id': 'track_id'})
df_track_artist = df_track_artist.rename(columns={'artist_id': 'artist_id', 'track_id': 'track_id'})
df_audio_features = df_audio_features.rename(columns={'id': 'track_id'})

In [5]:
print(df_tracks.head(2))
print(df_albums.head(2))
print(df_artists.head(2))

print(df_albums_tracks.head(2))
print(df_track_artist.head(2))

print(df_artist_genre.head(2))
print(df_audio_features.head(2))

                 track_id                      track  \
1  2g8HN35AnVGIk7B8yMucww  Big Poppa - 2005 Remaster   
3  4E5IFAXCob6QqZaJMTw5YN        Poppin' Them Thangs   

                                         preview_url  
1  https://p.scdn.co/mp3-preview/770e023eb0318270...  
3  https://p.scdn.co/mp3-preview/f3b556ced9657f89...  
                 album_id                                              album
0  2jKoVlU7VAmExKJ1Jh3w9P         Alkaholik (feat. Erik Sermon, J Ro & Tash)
1  4JYUDRtPZuVNi7FAnbHyux  Flava in Ya Ear Remix (feat. Notorious B.I.G.,...
                artist_id        artist
0  4tujQJicOnuZRLiBFdp3Ou        Xzibit
1  2VX0o9LDIVmKIgpnwdJpOJ  Erick Sermon
                 track_id                album_id
0  3HnrHGLE9u2MjHtdobfWl9  6os2Mv58OYnQClPf7B9E1s
1  4lDjkpUrpWlMFofIpzuExK  6os2Mv58OYnQClPf7B9E1s
                 track_id               artist_id
0  2jKoVlU7VAmExKJ1Jh3w9P  4tujQJicOnuZRLiBFdp3Ou
1  2jKoVlU7VAmExKJ1Jh3w9P  2VX0o9LDIVmKIgpnwdJpOJ


In [ ]:
df_spotify = df_tracks.merge(right=df_albums_tracks, how='inner', left_on='track_id', right_on='track_id')
df_spotify = df_spotify.merge(right=df_track_artist, how='inner', left_on='track_id', right_on='track_id')
df_spotify = df_spotify.merge(right=df_albums, how='inner', left_on='album_id', right_on='album_id')
df_spotify = df_spotify.merge(right=df_artists, how='inner', left_on='artist_id', right_on='artist_id')

df_spotify = df_spotify.merge(right=df_artist_genre, how='inner', left_on='artist_id', right_on='artist_id')
df_spotify = df_spotify.merge(right=df_audio_features, how='inner', left_on='track_id', right_on='track_id')


In [ ]:
df_spotify.info()
print(df_spotify)

In [ ]:
df_spotify.to_csv("aggregated_spotify_info.csv")

# Preprocessed load

In [ ]:
df_spotify = pd.read_csv("aggregated_spotify_info.csv")
df_spotify['artist'].apply(lambda x: x.lower())
df_spotify['album'].apply(lambda x: x.lower())
df_spotify['track'].apply(lambda x: x.lower())

In [ ]:
df_spotify.info()
print(df_spotify)

In [ ]:
df_private = pd.read_csv("tags.csv")
df_private = df_private[df_private['Artist'].notna()]
df_private = df_private[df_private['Album'].notna()]
df_private = df_private[df_private['Title'].notna()]
df_private = df_private.rename(columns={'Artist': 'artist', 'Album': 'album', 'Title': 'track'})

df_private['artist'].apply(lambda x: x.lower())
df_private['album'].apply(lambda x: x.lower())
df_private['track'].apply(lambda x: x.lower())

print(df_private)

# MERGE ON ALL

In [ ]:
df_merged = pd.merge(df_spotify, df_private,  how='inner', left_on=['track','artist','album'], right_on = ['track','artist','album'])

In [ ]:
df_merged.info()
print(df_merged)

In [ ]:
df_merged.to_csv("merged_datasets.csv")

# MERGE ON ARTIST AND TITLE

In [ ]:
df_merged = pd.merge(df_spotify, df_private,  how='inner', left_on=['track','artist'], right_on = ['track','artist'])